In [81]:
import torch
torch.cuda.is_available()  # Check if CUDA is available

True

In [96]:
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [97]:
df = pd.read_csv('WineQT.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [98]:
df.drop('Id', axis=1, inplace=True)

In [99]:
# Check for missing values
df.isna().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [100]:
np.unique(df.quality, return_counts=True)

(array([3, 4, 5, 6, 7, 8]), array([  6,  33, 483, 462, 143,  16]))

In [101]:
# Pytorch expects output to be in numerical indices to use croos entropy loss function!

quality_map = {
    3:'Low',
    4:'Low',
    5:"Average",
    6: "Average",
    7: "High",
    8: "High",
}
df.quality = df.quality.map(quality_map)
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,Average
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,Average
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,Average
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,Average
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,Average
...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,Average
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,Average
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,Average
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,Average


In [104]:
np.unique(df.quality,return_counts=True)

(array(['Average', 'High', 'Low'], dtype=object), array([945, 159,  39]))

In [128]:
label_encoder = LabelEncoder()
df.quality = label_encoder.fit_transform(df.quality)
print(np.unique( df.quality,return_counts=True))
y = df.quality

(array([0, 1, 2]), array([945, 159,  39]))


In [109]:
x = df.drop('quality',axis=1)
display(x,y,x.shape,y.shape)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2


0       0
1       0
2       0
3       0
4       0
       ..
1138    0
1139    0
1140    0
1141    0
1142    0
Name: quality, Length: 1143, dtype: int64

(1143, 11)

(1143,)

In [110]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_test,x_train,y_test,y_train)

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
158             6.8             0.610         0.04             1.5      0.057   
1081            6.9             0.840         0.21             4.1      0.074   
291             7.0             0.580         0.12             1.9      0.091   
538             7.8             0.480         0.68             1.7      0.415   
367            12.5             0.600         0.49             4.3      0.100   
...             ...               ...          ...             ...        ...   
66              5.0             1.020         0.04             1.4      0.045   
328            10.3             0.500         0.42             2.0      0.069   
67              6.8             0.775         0.00             3.0      0.102   
231            10.0             0.490         0.20            11.0      0.071   
966            11.6             0.475         0.40             1.4      0.091   

      free sulfur dioxide  

In [111]:
# Standardization of inputs that are fed to model
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print(x_train,x_test)
print(x_train.shape, x_test.shape)
# Convert to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)

print(x_train_tensor.shape, x_test_tensor.shape)
print(x_train_tensor, x_test_tensor)

[[ 0.1426802  -1.40273053  1.5096933  ... -0.09314751  0.56821234
   0.05661327]
 [ 0.96843134 -1.17920341  1.5096933  ... -1.07474038  0.4477703
   0.89445566]
 [ 0.3786091  -1.23508519  0.4833938  ...  0.56124774  0.20688622
   1.26683005]
 ...
 [-0.50612428 -1.01155806  0.32944888 ...  0.29948964 -0.33510296
   1.35992365]
 [-0.21121315  0.21784112  0.22681893 ... -0.15858703  0.20688622
  -0.87432272]
 [-0.44714205 -0.06156778  0.6886537  ... -0.35490561 -0.09421888
   1.26683005]] [[-0.86001763  0.44136825 -1.1586854  ...  0.69212679 -0.33510296
  -0.87432272]
 [-0.8010354   1.72664922 -0.28633082 ...  1.41196156  0.38754928
  -1.12257232]
 [-0.74205318  0.2737229  -0.7481656  ...  0.82300584 -1.0577552
   0.05661327]
 ...
 [-0.86001763  1.36341764 -1.3639453  ...  0.88844537 -0.57598704
   0.24280046]
 [ 1.02741357 -0.22921313 -0.3376458  ... -1.00930086  0.20688622
  -1.15360352]
 [ 1.97112917 -0.3130358   0.6886537  ... -1.59825658 -0.03399786
  -0.37782353]]
(914, 11) (229, 11

In [112]:
y_train

12      1
758     0
636     0
1109    0
743     0
       ..
1044    2
1095    0
1130    0
860     0
1126    0
Name: quality, Length: 914, dtype: int64

In [113]:
y_train_tensor  = torch.tensor(y_train.values,dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
print(y_train_tensor.shape, y_test_tensor.shape)
print(y_train_tensor, y_test_tensor)

torch.Size([914]) torch.Size([229])
tensor([1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        2., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 2., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 2., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 2., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 1., 1., 0., 0., 2., 2., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.,
    

In [117]:
import torch
import torch.nn as nn

# Multi-Class ANN Model (No softmax, ReLU in hidden layers)
class ANNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Hidden Layer 1 + ReLU
        x = self.relu(self.fc2(x))  # Hidden Layer 2 + ReLU
        x = self.fc3(x)             # Output Layer (raw logits)
        return x

In [118]:
input_size = x_train_tensor.shape[1]
print(input_size)

output_size = 3
print(output_size)

11
3


In [119]:
model = ANNModel(input_size, output_size)

In [120]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [123]:
import time 

num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(x_train_tensor)
    loss = criterion(outputs, y_train_tensor.long())  # Convert to long for CrossEntropyLoss
    
    _,predicted = torch.max(outputs, 1)  # Get the predicted class indices
    correct_predictions = (predicted == y_train_tensor.long()).sum().item()
    accuracy_var = correct_predictions/ len(y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        time.sleep(0.5)  # Simulate some delay for better readability in output
        print(f'Epoch [{epoch+1}/{num_epochs}]:')
        print(f"Loss: {loss.item():.4f}")
        print(f"Accuracy: {accuracy_var:.4f}")

Epoch [10/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [20/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [30/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [40/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [50/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [60/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [70/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [80/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [90/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [100/1000]:
Loss: 0.0006
Accuracy: 1.0000
Epoch [110/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [120/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [130/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [140/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [150/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [160/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [170/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [180/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [190/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [200/1000]:
Loss: 0.0005
Accuracy: 1.0000
Epoch [210/1000]:
Loss: 0.0005
Accuracy: 1.0000
E

In [125]:
model.eval()
with torch.inference_mode():
    outputs = model(x_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get the predicted class indices
    accuracy = accuracy_score(y_test, predicted)
    print(predicted)
    
    predicted_tensor = predicted.clone().detach()
    loss = criterion(outputs, predicted_tensor)
    print(f'loss: {loss.item():.4f}, accuracy: {accuracy:.4f}')


tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        1, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
loss: 0.0285, accuracy: 0.8603


### pickle all the files along with model

In [127]:
import pickle


torch.save(model.state_dict(), 'model.pth')

with open('scaler.pickle', 'wb') as f:
    pickle.dump(scaler, f)
    f.close()